 ### 03 - BASELINE MODEL - LEAD-3
 ----

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
pip install --upgrade nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.1 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


In [ ]:
import nltk
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from datasets import Dataset

In [ ]:
df = pd.read_csv('/content/df_cleaned_sample1.csv', on_bad_lines='skip', engine='python')
dataset_bart = Dataset.from_pandas(df)

In [ ]:
dataset_bart.shape # check dimensions

{'train': (6044, 2), 'test': (1512, 2)}

In [ ]:
dataset_bart[1] # check first entry

{'highlights': 'the arsenal striker picked up a knock at london colney on tuesday  but roy hodgson has refused to rule him out of the wembley friendly  the gunners signed welbeck in a £16million deadline day deal  england face switzerland in their first euro 2016 qualifier on monday .',
 'article': "  follow   joe strange . arsenal forward danny welbeck picked up an ankle knock in england training at london colney on tuesday but could still be fit enough to play some part against norway. the three lions welcome per mathias hogmo's side to wembley on wednesday for their first game since being knocked out in the group stage of this summer's world cup. roy hodgson has refused to rule welbeck out of the friendly, but his injury means that he is likely to begin the match on the bench. video scroll down to watch welbeck's part in one touch training goal . knock  danny welbeck injured his ankle in england training on tuesday but could still play against norway . calm before the storm  welbeck

In [ ]:
# split for -  (80% train, 20% test)
dataset_bart = dataset_bart.train_test_split(test_size=0.2)
train_dataset_bart = dataset_bart['train']
test_dataset_bart = dataset_bart['test']

In [ ]:
#validation and train set 

#train_val_split = train_dataset_bart.train_test_split(test_size=0.2)  # 20% of training data for validation
#train_dataset_bart = train_val_split['train']
#validation_dataset_bart = train_val_split['test']

In [ ]:
dataset_bart.keys() #check that the dataset has been split properly

dict_keys(['train', 'test'])

In [ ]:
# LEAD 3 AND TOKENIZER

from nltk.tokenize import sent_tokenize

def lead_3_func(text):
    sentences = sent_tokenize(text)
    three_lines = sentences[0:3]
    result = "\n".join(three_lines)
    return result


print(lead_3_func(dataset_bart["train"][1]["article"]))

# https://huggingface.co/learn/nlp-course/chapter7/5?fw=pt#summarization

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 899.4 kB/s eta 0:00:00


In [ ]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ed9a0158f05bc420cf601cdc758bcc500bed69dcadb6fc7a961bfd959ce6e0b4
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# imports 

import evaluate

rouge_score = evaluate.load("rouge") #load rouge metric


In [ ]:
# evaluation - rouge metric

In [ ]:
def evaluation(dataset, metric):
    summaries = [lead3(text) for text in dataset["article"]]
    return metric.compute(predictions=summaries, references=dataset["highlights"])

In [ ]:
# Perform evaluation
score = evaluation(validation_dataset_bart, rouge_score)

rouge_submetrics = ["rouge1", "rouge2", "rougeL", "rougeLsum"] # rouge metrics

rouge_scores = {rn: round(score[rn] * 100, 2) for rn in rouge_submetrics} #dictionary with scores


rouge_scores #show the scores

# documentation for rouge metric: https://huggingface.co/spaces/evaluate-metric/rouge
# https://huggingface.co/learn/nlp-course/chapter7/5?fw=pt#summarization

{'rouge1': 34.88, 'rouge2': 14.6, 'rougeL': 22.37, 'rougeLsum': 26.33}